In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', 1000)

In [3]:
# Defining directories
dir_path = Path.cwd().parent
prep_folder = dir_path.joinpath("data/prep")
s2_path = prep_folder.joinpath("sent2.parquet")

processed_folder = dir_path.joinpath("data/processed")
if not processed_folder.exists():
    processed_folder.mkdir(parents=True)

In [4]:
df = pd.read_parquet(s2_path)
df

,date,long,lat,species_names,B11,B02,B12,B08,B04,B03
0,2019-08-23,76.98762370000003,31.571770300000026,Pinus roxburghii,2234.0,376.0,1084.0,3340.0,468.0,739.0
1,2019-08-23,76.98770870000004,31.571733300000062,Pinus roxburghii,2234.0,240.0,1084.0,3374.0,248.0,496.0
2,2019-08-23,76.98786920000003,31.571645600000068,Pinus roxburghii,1826.0,269.0,823.0,3458.0,216.0,525.0
3,2019-08-23,76.98792120000007,31.572662200000025,Pinus roxburghii,2037.0,345.0,971.0,3754.0,359.0,699.0
4,2019-08-23,76.98797090000005,31.57163410000004,Pinus roxburghii,1826.0,247.0,823.0,3248.0,245.0,529.0
...,...,...,...,...,...,...,...,...,...,...
13193487,2021-11-25,78.12497830000007,31.25865170000003,Rhododendron arboreum,1781.0,1115.0,1436.0,1827.0,1196.0,1221.0
13193488,2021-11-25,78.12522000000007,31.258880000000033,Rhododendron arboreum,1657.0,1175.0,1431.0,1704.0,1216.0,1230.0
13193489,2021-11-25,78.12614830000007,31.258660000000077,Rhododendron arboreum,1678.0,1113.0,1389.0,1954.0,1222.0,1175.0
13193490,2021-11-25,78.12618260000005,31.25882800000005,Rhododendron arboreum,1966.0,1104.0,1481.0,2143.0,1240.0,1219.0


In [5]:
for band in ["B11", "B02", "B12", "B08", "B04", "B03"]:
    print(df[band].isna().value_counts())

B11
False    13193492
Name: count, dtype: int64
B02
False    13193388
True          104
Name: count, dtype: int64
B12
False    13193492
Name: count, dtype: int64
B08
False    13193492
Name: count, dtype: int64
B04
False    13193463
True           29
Name: count, dtype: int64
B03
False    13193459
True           33
Name: count, dtype: int64


In [6]:
# Extracting month and year
df["year"] = pd.to_datetime(df["date"]).dt.year
df["month"] = pd.to_datetime(df["date"]).dt.month
df.drop('date', axis=1, inplace=True)

# scaling the band values by a factor 0.0001
for band in ["B11", "B02", "B12", "B08", "B04", "B03"]:
    df[band] = df[band] * 0.0001

In [7]:
for band in ["B11", "B02", "B12", "B08", "B04", "B03"]:
    print(df[band].isna().value_counts())

B11
False    13193492
Name: count, dtype: int64
B02
False    13193388
True          104
Name: count, dtype: int64
B12
False    13193492
Name: count, dtype: int64
B08
False    13193492
Name: count, dtype: int64
B04
False    13193463
True           29
Name: count, dtype: int64
B03
False    13193459
True           33
Name: count, dtype: int64


In [8]:
def make_s2_indices(df):

    #### SENTINEL 2

    ## NDVI
    df["NDVI"] = (df["B08"] - df["B04"]) / (df["B08"] + df["B04"])
    temp_df = pd.DataFrame(df["NDVI"])

    ## Atmospherically Resistant Vegetation Index 2
    df["ARVI2"] = -0.18 + 1.17 * df["NDVI"]

    ## Blue-wide dynamic range vegetation index
    df["BWDRVI"] = (0.1 * (df["B08"] - df["B02"])) / (0.1 * (df["B08"] + df["B02"]))

    ## Chlorophyll vegetation index
    df["CVI"] = df["B08"] * (df["B04"] / (df["B03"].pow(2)))

    ## Corrected Transformed Vegetation Index
    temp_df["ndvi+0.5"] = temp_df["NDVI"] + 0.5
    temp_df["ctvi1"] = (temp_df["ndvi+0.5"]) / (temp_df["ndvi+0.5"].abs())
    temp_df["ctvi2"] = np.sqrt(temp_df["ndvi+0.5"].abs())
    df["CTVI"] = temp_df["ctvi1"] * temp_df["ctvi2"]

    ## Enhanced Vegetation Index 2 -2
    df["EVI2"] = 2.5 * ((df["B08"] - df["B04"]) / (df["B08"] + (2.4 * df["B04"]) + 1))

    ## Global Vegetation Moisture Index
    temp_df['gvmi_num'] = (df['B08'] + 0.1) - (df['B12'] + 0.02)
    temp_df['gvmi_den'] = (df['B08'] + 0.1) + (df['B12'] + 0.02)
    df['GVMI'] = temp_df['gvmi_num']/temp_df['gvmi_den']

    ## Modified Soil Adjusted Vegetation Index hyper
    temp_df['hyper_first_term'] = 2 * df['B08'] + 1
    temp_df['hyper_second'] = np.sqrt(temp_df['hyper_first_term'].pow(2) - (8 * (df['B08'] - df['B04'])))
    df['MSVAIhyper'] = 0.5 * (temp_df['hyper_first_term'] - temp_df['hyper_second'])

    ## Modified Triangular Vegetation Index 2
    temp_df['mtvi2_num'] = 1.5 * (1.2 * (df['B08'] - df['B03']) - 2.5 *(df['B04'] - df['B03']))
    temp_df['mtvi2_den'] = np.sqrt((2 * df['B08'] + 1).pow(2) - (6 * df['B08'] - 5*(np.sqrt(df['B04']))) - 0.5)
    df['MTVI2'] = temp_df['mtvi2_num']/temp_df['mtvi2_den']
    # MTVI2 = 1.5*(1.2 * (NIR - Green) - 2.5 * (Red - Green))√((2 * NIR + 1)²-(6 * NIR - 5√(Red)) - 0.5)


    ## Normalized Difference NIR/MIR Modified Normalized Difference Vegetation Index
    df['MNDVI'] = (df['B08'] - df['B12'])/(df['B08'] + df['B12'])

    ## Optimized Soil Adjusted Vegetation Index
    temp_df['Y'] = 0.16
    temp_df['osavi_num'] = df['B08'] - df['B04']
    temp_df['osavi_den'] = df['B08'] + df['B04'] + temp_df['Y']
    df['OSAVI'] = (temp_df['Y'] + 1) * (temp_df['osavi_num']/temp_df['osavi_den'])

    ## Perpendicular Vegetation Index
    pvi_a = 0.149
    pvi_ar = 0.374
    pvi_b = 0.735

    temp_df['pvi_first'] = 1/(np.sqrt(1 + (pvi_a*pvi_a)))
    temp_df['pvi_second'] = df['B08'] - pvi_ar - pvi_b
    df['PVI'] = temp_df['pvi_first'] * temp_df['pvi_second']

    ## Soil and Atmospherically Resistant Vegetation Index
    y = 0.735
    Rr = 0.740
    L = 0.487
    RB = 0.560

    temp_df['sarvi_num'] = df['B08'] - (Rr - y * (RB-Rr))
    temp_df['sarvi_den'] = (df['B08'] + -(Rr - y * (RB-Rr))) + L
    df['SARVI'] = (1+L) * (temp_df['sarvi_num'] / temp_df['sarvi_den'])

    ## Specific Leaf Area Vegetation Index
    df['SLAVI'] = df['B08'] / (df['B04'] + df['B12'])

    ## Transformed Soil Adjusted Vegetation Index 2
    tsavi2_a = 0.419
    tsavi2_b = 0.787
    temp_df['tsavi2_num'] = (tsavi2_a * df['B08']) - (tsavi2_a * df['B04']) - tsavi2_b
    temp_df['tsavi2_den'] = df['B04'] + (tsavi2_a * df['B08']) - (tsavi2_a * tsavi2_b)
    df['TSAVI2'] = temp_df['tsavi2_num'] / temp_df['tsavi2_den']

    ## Weighted Difference Vegetation Index
    wdvi_a = 0.752
    df['WDVI'] = df['B08'] - (wdvi_a * df['B04'])

    ## Wide Dynamic Range Vegetation Index
    df['WDRVI'] = 0.1*(df['B08'] - df['B04'])/(0.1*(df['B08'] + df['B04']))

    return df

df_indices = make_s2_indices(df)
df_indices

,long,lat,species_names,B11,B02,B12,B08,B04,B03,year,month,NDVI,ARVI2,BWDRVI,CVI,CTVI,EVI2,GVMI,MSVAIhyper,MTVI2,MNDVI,OSAVI,PVI,SARVI,SLAVI,TSAVI2,WDVI,WDRVI
0,76.98762370000003,31.571770300000026,Pinus roxburghii,0.2234,0.0376,0.1084,0.3340,0.0468,0.0739,2019,8,-0.224524,-0.442693,0.797632,2.862223,0.524858,0.496432,0.543385,0.485925,0.488624,0.509946,0.616036,-0.766538,15.603355,2.152062,4.661752,0.298806,0.754202
1,76.98770870000004,31.571733300000062,Pinus roxburghii,0.2234,0.0240,0.1084,0.3374,0.0248,0.0496,2019,8,-0.333333,-0.570000,0.867183,3.401210,0.408248,0.559445,0.546129,0.561649,0.591280,0.513683,0.694401,-0.763175,16.605351,2.533033,4.010337,0.318750,0.863059
2,76.98786920000003,31.571645600000068,Pinus roxburghii,0.1826,0.0269,0.0823,0.3458,0.0216,0.0525,2019,8,-0.417004,-0.667895,0.855648,2.709943,0.288090,0.579906,0.626710,0.587000,0.636986,0.615510,0.713068,-0.754867,19.820392,3.328200,3.988417,0.329557,0.882417
3,76.98792120000007,31.572662200000025,Pinus roxburghii,0.2037,0.0345,0.0971,0.3754,0.0359,0.0699,2019,8,-0.321361,-0.555992,0.831666,2.758255,0.422657,0.580715,0.604726,0.579892,0.603412,0.588995,0.689340,-0.725590,74.635384,2.822556,4.721350,0.348403,0.825432
4,76.98797090000005,31.57163410000004,Pinus roxburghii,0.1826,0.0247,0.0823,0.3248,0.0245,0.0529,2019,8,-0.366925,-0.609302,0.858655,2.843615,0.364794,0.542606,0.611838,0.542497,0.580178,0.595677,0.683974,-0.775637,13.456736,3.041199,3.908532,0.306376,0.859719
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13193487,78.12497830000007,31.25865170000003,Rhododendron arboreum,0.1781,0.1115,0.1436,0.1827,0.1196,0.1221,2021,11,-0.010343,-0.192102,0.242012,1.465677,0.699755,0.107332,0.266861,0.099708,0.083817,0.119828,0.158330,-0.916186,5.061378,0.694149,5.692750,0.092761,0.208733
13193488,78.12522000000007,31.258880000000033,Rhododendron arboreum,0.1657,0.1175,0.1431,0.1704,0.1216,0.1230,2021,11,-0.005724,-0.186697,0.183744,1.369597,0.703048,0.083434,0.247520,0.077242,0.063742,0.087081,0.125239,-0.928351,4.856795,0.643748,5.605284,0.078957,0.167123
13193489,78.12614830000007,31.258660000000077,Rhododendron arboreum,0.1678,0.1113,0.1389,0.1954,0.1222,0.1175,2021,11,0.019608,-0.157059,0.274209,1.729498,0.720838,0.122928,0.300462,0.114727,0.086477,0.169010,0.177789,-0.903624,5.300423,0.748372,6.017877,0.103506,0.230479
13193490,78.12618260000005,31.25882800000005,Rhododendron arboreum,0.1966,0.1104,0.1481,0.2143,0.1240,0.1219,2021,11,0.008540,-0.170008,0.319988,1.788284,0.713120,0.149315,0.303068,0.140171,0.111598,0.182671,0.210211,-0.884931,5.721906,0.787578,6.460468,0.121052,0.266923


In [10]:
for col in df_indices.columns:
    print(df_indices[col].isna().value_counts())


long
False    13193492
Name: count, dtype: int64
lat
False    13193492
Name: count, dtype: int64
species_names
False    13193492
Name: count, dtype: int64
B11
False    13193492
Name: count, dtype: int64
B02
False    13193388
True          104
Name: count, dtype: int64
B12
False    13193492
Name: count, dtype: int64
B08
False    13193492
Name: count, dtype: int64
B04
False    13193463
True           29
Name: count, dtype: int64
B03
False    13193459
True           33
Name: count, dtype: int64
year
False    13193492
Name: count, dtype: int64
month
False    13193492
Name: count, dtype: int64
NDVI
False    13193430
True           62
Name: count, dtype: int64
ARVI2
False    13193430
True           62
Name: count, dtype: int64
BWDRVI
False    13193388
True          104
Name: count, dtype: int64
CVI
False    13193430
True           62
Name: count, dtype: int64
CTVI
False    13193421
True           71
Name: count, dtype: int64
EVI2
False    13193463
True           29
Name: count, dtype: int64


### Making Monthly Medians

Monthly medians, as expected, have misisng values afte rmedian computations

In [11]:
index_cols = ["long", "lat", "year", "month", "species_names"]
#making monthlu medians
df_month_medians = (
    df_indices.groupby(index_cols)
    .median()
    .reset_index()
)

df_month_medians

,long,lat,year,month,species_names,B11,B02,B12,B08,B04,B03,NDVI,ARVI2,BWDRVI,CVI,CTVI,EVI2,GVMI,MSVAIhyper,MTVI2,MNDVI,OSAVI,PVI,SARVI,SLAVI,TSAVI2,WDVI,WDRVI
0,75.66669580000007,32.25107380000003,2018,2,Bambusoideae spp.,0.26240,0.14080,0.20730,0.28920,0.16830,0.16120,0.021548,-0.154789,0.345116,1.873063,0.722183,0.178517,0.262612,0.171918,0.134151,0.164955,0.227116,-0.810849,9.022578,0.769968,18.281425,0.162638,0.264262
1,75.66669580000007,32.25107380000003,2018,3,Bambusoideae spp.,0.28310,0.13900,0.22460,0.31860,0.17640,0.17100,0.015544,-0.161813,0.392483,1.921994,0.718014,0.204080,0.262364,0.197548,0.159463,0.173049,0.251835,-0.781770,12.344106,0.794514,36.628039,0.185947,0.287273
2,75.66669580000007,32.25107380000003,2018,4,Bambusoideae spp.,0.28970,0.15440,0.23560,0.34180,0.17550,0.17720,-0.004820,-0.185639,0.377670,1.910389,0.703690,0.235820,0.266992,0.228589,0.196250,0.183928,0.284819,-0.758823,18.134563,0.831428,64.981728,0.209824,0.321477
3,75.66669580000007,32.25107380000003,2018,5,Bambusoideae spp.,0.32590,0.21600,0.26770,0.32420,0.23600,0.23600,0.000000,-0.180000,0.200296,1.373729,0.707107,0.116630,0.191740,0.115042,0.096600,0.095455,0.142061,-0.776231,13.339193,0.643637,-17.821364,0.146728,0.157444
4,75.66669580000007,32.25107380000003,2018,6,Bambusoideae spp.,0.32320,0.16480,0.26840,0.34100,0.19560,0.19280,0.007209,-0.171565,0.348359,1.794358,0.712186,0.200780,0.209213,0.195647,0.162253,0.119134,0.242125,-0.759614,17.833930,0.734914,-83.208503,0.193909,0.270965
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9478188,78.12624330000006,31.258713300000068,2024,8,Rhododendron arboreum,0.21590,0.11680,0.16170,0.33150,0.11900,0.12700,-0.032520,-0.218049,0.478920,2.445812,0.683725,0.328520,0.407371,0.315368,0.281402,0.344282,0.403767,-0.769010,14.947390,1.180976,9.713553,0.242012,0.471698
9478189,78.12624330000006,31.258713300000068,2024,9,Rhododendron arboreum,0.19565,0.11470,0.14720,0.32305,0.11240,0.14260,-0.118253,-0.318356,0.475040,1.793129,0.617828,0.330430,0.433184,0.316796,0.313829,0.373414,0.409749,-0.777368,13.982224,1.244150,8.548427,0.238525,0.482899
9478190,78.12624330000006,31.258713300000068,2024,10,Rhododendron arboreum,0.18175,0.09995,0.12895,0.22385,0.10470,0.10710,-0.011375,-0.193309,0.382070,2.044298,0.699005,0.201748,0.369414,0.189244,0.160179,0.268025,0.282372,-0.875485,6.000114,0.959065,5.628229,0.145116,0.361852
9478191,78.12624330000006,31.258713300000068,2024,11,Rhododendron arboreum,0.18195,0.12695,0.14900,0.21795,0.13615,0.13635,-0.000657,-0.180768,0.266684,1.596637,0.706638,0.132369,0.305819,0.124763,0.101885,0.187783,0.184541,-0.881321,5.817910,0.764326,7.375042,0.115565,0.230987


### Making Season Medians 

No missing values in seasonal medians

In [12]:
# creating seasons
season_index_cols = ["long", "lat", "season", "species_names"]


conds = [df_month_medians["month"].isin([3, 4, 5]), df_month_medians["month"].isin([10, 11, 12])]
opts = ["summer", "winter"]


df_month_medians["season"] = np.select(conds, opts, default="NA")
df_season_medians = df_month_medians[~(df_month_medians["season"] == "NA")]
df_season_medians["season"] = df_season_medians["season"] + df_month_medians["year"].astype(str) 


df_season_medians = df_season_medians.groupby(season_index_cols).median().reset_index()
df_season_medians

/tmp/ipykernel_1526990/2127264070.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_season_medians["season"] = df_season_medians["season"] + df_month_medians["year"].astype(str)


,long,lat,season,species_names,year,month,B11,B02,B12,B08,B04,B03,NDVI,ARVI2,BWDRVI,CVI,CTVI,EVI2,GVMI,MSVAIhyper,MTVI2,MNDVI,OSAVI,PVI,SARVI,SLAVI,TSAVI2,WDVI,WDRVI
0,75.66669580000007,32.25107380000003,summer2018,Bambusoideae spp.,2018.0,4.0,0.28970,0.15440,0.23560,0.32420,0.17640,0.17720,0.000000,-0.180000,0.377670,1.910389,0.707107,0.204080,0.262364,0.197548,0.159463,0.173049,0.251835,-0.776231,13.339193,0.794514,36.628039,0.185947,0.287273
1,75.66669580000007,32.25107380000003,summer2019,Bambusoideae spp.,2019.0,4.5,0.25885,0.11280,0.19540,0.29230,0.13280,0.12965,0.021650,-0.154670,0.477402,2.683532,0.722090,0.249000,0.295781,0.237530,0.196948,0.200573,0.325497,-0.807782,18.051811,0.916986,-36.203756,0.192434,0.400403
2,75.66669580000007,32.25107380000003,summer2020,Bambusoideae spp.,2020.0,4.0,0.27645,0.14465,0.21670,0.34525,0.16110,0.16560,-0.012700,-0.194859,0.407303,2.025796,0.697924,0.264676,0.305009,0.255691,0.223837,0.226889,0.318257,-0.755411,170.718932,0.911091,140.143094,0.224103,0.360856
3,75.66669580000007,32.25107380000003,summer2021,Bambusoideae spp.,2021.0,4.5,0.29260,0.15000,0.23995,0.31960,0.18160,0.17520,0.017977,-0.158966,0.361201,1.895898,0.719566,0.196552,0.234981,0.190535,0.152337,0.142406,0.242141,-0.780781,12.514366,0.758327,55.341681,0.183037,0.275399
4,75.66669580000007,32.25107380000003,summer2022,Bambusoideae spp.,2022.0,4.0,0.30780,0.15360,0.23400,0.32590,0.19020,0.17780,0.032240,-0.142279,0.365854,1.957255,0.729548,0.195165,0.268881,0.189574,0.145198,0.182296,0.238824,-0.774549,13.678397,0.789678,123.528038,0.185050,0.270070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2084693,78.12624330000006,31.258713300000068,winter2020,Rhododendron arboreum,2020.0,11.0,0.15530,0.15385,0.13620,0.19710,0.13250,0.14450,-0.043145,-0.230480,0.152081,1.446515,0.675856,0.118107,0.314275,0.110411,0.097110,0.186134,0.170207,-0.901943,5.308156,0.749256,6.494663,0.102273,0.220117
2084694,78.12624330000006,31.258713300000068,winter2021,Rhododendron arboreum,2021.0,11.0,0.16685,0.15805,0.13680,0.19795,0.11840,0.13760,-0.000763,-0.180893,0.112399,1.365192,0.706557,0.095915,0.238653,0.092677,0.069931,0.115905,0.128573,-0.901102,4.632840,0.663562,5.083158,0.108913,0.157806
2084695,78.12624330000006,31.258713300000068,winter2022,Rhododendron arboreum,2022.0,11.0,0.16190,0.13255,0.12955,0.17620,0.11455,0.12110,-0.021048,-0.204626,0.168492,1.384757,0.692029,0.106228,0.297474,0.098332,0.088290,0.152568,0.158703,-0.922615,4.950380,0.721901,5.386990,0.090058,0.212149
2084696,78.12624330000006,31.258713300000068,winter2023,Rhododendron arboreum,2023.0,11.0,0.16420,0.09895,0.13355,0.17600,0.10340,0.10235,0.005578,-0.173474,0.280555,1.770120,0.710985,0.127468,0.279856,0.117653,0.093834,0.130750,0.191703,-0.922813,4.946996,0.737550,4.958911,0.098243,0.259935


### Splitting Testing and Training
All data from 2024 will be assigned for testing and data from 2018 - 2023 (6 years) will be used for training.

In [13]:
sent2_train_season_medians = df_season_medians[df_season_medians['year']!=2024]
sent2_test_season_medians = df_season_medians[df_season_medians['year']==2024]


sent2_train_season_medians = sent2_train_season_medians.drop(["year", "month"], axis=1)
sent2_test_season_medians = sent2_test_season_medians.drop(["year", "month"], axis=1)

sent2_test_season_medians

,long,lat,season,species_names,B11,B02,B12,B08,B04,B03,NDVI,ARVI2,BWDRVI,CVI,CTVI,EVI2,GVMI,MSVAIhyper,MTVI2,MNDVI,OSAVI,PVI,SARVI,SLAVI,TSAVI2,WDVI,WDRVI
6,75.66669580000007,32.25107380000003,summer2024,Bambusoideae spp.,0.29080,0.14230,0.22880,0.33480,0.16720,0.16640,0.002398,-0.177194,0.405837,2.065170,0.708800,0.209016,0.254991,0.203063,0.154652,0.155412,0.253745,-0.765746,15.826980,0.775898,37.068751,0.195003,0.285961
13,75.66669580000007,32.25107380000003,winter2024,Bambusoideae spp.,0.25830,0.15780,0.19190,0.34770,0.15900,0.17030,-0.034315,-0.220149,0.375668,1.906218,0.682411,0.272798,0.354750,0.263581,0.239437,0.285022,0.328322,-0.752987,20.746814,1.000841,28.242038,0.228759,0.372410
20,75.66676390000003,32.25073910000003,summer2024,Bambusoideae spp.,0.31710,0.13660,0.24580,0.34540,0.15810,0.16480,-0.022566,-0.206402,0.421399,2.035980,0.690966,0.253276,0.221278,0.245149,0.204307,0.133574,0.305154,-0.755262,19.636599,0.799167,29.353192,0.218914,0.345016
27,75.66676390000003,32.25073910000003,winter2024,Bambusoideae spp.,0.27550,0.15180,0.19890,0.33450,0.15340,0.16380,-0.032787,-0.218361,0.375694,1.912466,0.683530,0.283844,0.337335,0.273260,0.232773,0.262741,0.336195,-0.766043,15.742295,0.971819,15.439635,0.226362,0.372159
34,75.66686810000004,32.25026770000005,summer2024,Bambusoideae spp.,0.32890,0.14420,0.25470,0.33860,0.16220,0.17200,-0.020070,-0.203482,0.388559,1.992846,0.692770,0.219832,0.221538,0.213420,0.181254,0.120755,0.271958,-0.761988,16.993831,0.767839,57.208149,0.201435,0.317657
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2084669,78.12614830000007,31.258660000000077,winter2024,Rhododendron arboreum,0.18220,0.13045,0.14535,0.22275,0.13635,0.13045,0.007579,-0.171132,0.265971,1.809724,0.712426,0.142331,0.318520,0.134378,0.101822,0.206402,0.196709,-0.876573,5.946358,0.790678,7.559093,0.122165,0.244736
2084676,78.12618260000005,31.25882800000005,summer2024,Rhododendron arboreum,0.22280,0.17600,0.17290,0.27855,0.15610,0.17300,-0.053842,-0.242995,0.223813,1.467303,0.667826,0.181560,0.326215,0.174169,0.169701,0.235362,0.232748,-0.821382,8.518848,0.841399,13.304565,0.159283,0.273971
2084683,78.12618260000005,31.25882800000005,winter2024,Rhododendron arboreum,0.20700,0.12365,0.15390,0.24995,0.14150,0.13975,-0.006341,-0.187419,0.338736,1.827620,0.702590,0.170599,0.303257,0.162164,0.131382,0.198699,0.228187,-0.849670,6.837432,0.809950,8.896813,0.143542,0.277155
2084690,78.12624330000006,31.258713300000068,summer2024,Rhododendron arboreum,0.22115,0.18330,0.17425,0.30525,0.15660,0.16970,-0.053403,-0.242482,0.234178,1.477287,0.668270,0.194787,0.355068,0.188081,0.183950,0.275292,0.238455,-0.794974,6.944189,0.878359,26.076134,0.176884,0.272490


In [16]:
sent2_train_season_medians

,long,lat,season,species_names,B11,B02,B12,B08,B04,B03,NDVI,ARVI2,BWDRVI,CVI,CTVI,EVI2,GVMI,MSVAIhyper,MTVI2,MNDVI,OSAVI,PVI,SARVI,SLAVI,TSAVI2,WDVI,WDRVI
0,75.66669580000007,32.25107380000003,summer2018,Bambusoideae spp.,0.28970,0.15440,0.23560,0.32420,0.17640,0.17720,0.000000,-0.180000,0.377670,1.910389,0.707107,0.204080,0.262364,0.197548,0.159463,0.173049,0.251835,-0.776231,13.339193,0.794514,36.628039,0.185947,0.287273
1,75.66669580000007,32.25107380000003,summer2019,Bambusoideae spp.,0.25885,0.11280,0.19540,0.29230,0.13280,0.12965,0.021650,-0.154670,0.477402,2.683532,0.722090,0.249000,0.295781,0.237530,0.196948,0.200573,0.325497,-0.807782,18.051811,0.916986,-36.203756,0.192434,0.400403
2,75.66669580000007,32.25107380000003,summer2020,Bambusoideae spp.,0.27645,0.14465,0.21670,0.34525,0.16110,0.16560,-0.012700,-0.194859,0.407303,2.025796,0.697924,0.264676,0.305009,0.255691,0.223837,0.226889,0.318257,-0.755411,170.718932,0.911091,140.143094,0.224103,0.360856
3,75.66669580000007,32.25107380000003,summer2021,Bambusoideae spp.,0.29260,0.15000,0.23995,0.31960,0.18160,0.17520,0.017977,-0.158966,0.361201,1.895898,0.719566,0.196552,0.234981,0.190535,0.152337,0.142406,0.242141,-0.780781,12.514366,0.758327,55.341681,0.183037,0.275399
4,75.66669580000007,32.25107380000003,summer2022,Bambusoideae spp.,0.30780,0.15360,0.23400,0.32590,0.19020,0.17780,0.032240,-0.142279,0.365854,1.957255,0.729548,0.195165,0.268881,0.189574,0.145198,0.182296,0.238824,-0.774549,13.678397,0.789678,123.528038,0.185050,0.270070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2084692,78.12624330000006,31.258713300000068,winter2019,Rhododendron arboreum,0.16170,0.11845,0.13325,0.20275,0.12440,0.12090,-0.027404,-0.212063,0.263483,1.731773,0.687456,0.130475,0.312071,0.122100,0.094354,0.184026,0.186600,-0.896355,5.455463,0.767643,6.274160,0.109201,0.239591
2084693,78.12624330000006,31.258713300000068,winter2020,Rhododendron arboreum,0.15530,0.15385,0.13620,0.19710,0.13250,0.14450,-0.043145,-0.230480,0.152081,1.446515,0.675856,0.118107,0.314275,0.110411,0.097110,0.186134,0.170207,-0.901943,5.308156,0.749256,6.494663,0.102273,0.220117
2084694,78.12624330000006,31.258713300000068,winter2021,Rhododendron arboreum,0.16685,0.15805,0.13680,0.19795,0.11840,0.13760,-0.000763,-0.180893,0.112399,1.365192,0.706557,0.095915,0.238653,0.092677,0.069931,0.115905,0.128573,-0.901102,4.632840,0.663562,5.083158,0.108913,0.157806
2084695,78.12624330000006,31.258713300000068,winter2022,Rhododendron arboreum,0.16190,0.13255,0.12955,0.17620,0.11455,0.12110,-0.021048,-0.204626,0.168492,1.384757,0.692029,0.106228,0.297474,0.098332,0.088290,0.152568,0.158703,-0.922615,4.950380,0.721901,5.386990,0.090058,0.212149


### Widening the frame

#### Widening Train

In [17]:
sent2_train_season_medians_wide = sent2_train_season_medians.pivot(index=['long', 'lat', 'species_names'], columns=['season']).reset_index()
new_cols = ["".join(x) for x in sent2_train_season_medians_wide.columns]
sent2_train_season_medians_wide.columns = new_cols
sent2_train_season_medians_wide

,long,lat,species_names,B11summer2018,B11summer2019,B11summer2020,B11summer2021,B11summer2022,B11summer2023,B11winter2018,B11winter2019,B11winter2020,B11winter2021,B11winter2022,B11winter2023,B02summer2018,B02summer2019,B02summer2020,B02summer2021,B02summer2022,B02summer2023,B02winter2018,B02winter2019,B02winter2020,B02winter2021,B02winter2022,B02winter2023,B12summer2018,B12summer2019,B12summer2020,B12summer2021,B12summer2022,B12summer2023,B12winter2018,B12winter2019,B12winter2020,B12winter2021,B12winter2022,B12winter2023,B08summer2018,B08summer2019,B08summer2020,B08summer2021,B08summer2022,B08summer2023,B08winter2018,B08winter2019,B08winter2020,B08winter2021,B08winter2022,B08winter2023,B04summer2018,B04summer2019,B04summer2020,B04summer2021,B04summer2022,B04summer2023,B04winter2018,B04winter2019,B04winter2020,B04winter2021,B04winter2022,B04winter2023,B03summer2018,B03summer2019,B03summer2020,B03summer2021,B03summer2022,B03summer2023,B03winter2018,B03winter2019,B03winter2020,B03winter2021,B03winter2022,B03winter2023,NDVIsummer2018,NDVIsummer2019,NDVIsummer2020,NDVIsummer2021,NDVIsummer2022,NDVIsummer2023,NDVIwinter2018,NDVIwinter2019,NDVIwinter2020,NDVIwinter2021,NDVIwinter2022,NDVIwinter2023,ARVI2summer2018,ARVI2summer2019,ARVI2summer2020,ARVI2summer2021,ARVI2summer2022,ARVI2summer2023,ARVI2winter2018,ARVI2winter2019,ARVI2winter2020,ARVI2winter2021,ARVI2winter2022,ARVI2winter2023,BWDRVIsummer2018,BWDRVIsummer2019,BWDRVIsummer2020,BWDRVIsummer2021,BWDRVIsummer2022,BWDRVIsummer2023,BWDRVIwinter2018,BWDRVIwinter2019,BWDRVIwinter2020,BWDRVIwinter2021,BWDRVIwinter2022,BWDRVIwinter2023,CVIsummer2018,CVIsummer2019,CVIsummer2020,CVIsummer2021,CVIsummer2022,CVIsummer2023,CVIwinter2018,CVIwinter2019,CVIwinter2020,CVIwinter2021,CVIwinter2022,CVIwinter2023,CTVIsummer2018,CTVIsummer2019,CTVIsummer2020,CTVIsummer2021,CTVIsummer2022,CTVIsummer2023,CTVIwinter2018,CTVIwinter2019,CTVIwinter2020,CTVIwinter2021,CTVIwinter2022,CTVIwinter2023,EVI2summer2018,EVI2summer2019,EVI2summer2020,EVI2summer2021,EVI2summer2022,EVI2summer2023,EVI2winter2018,EVI2winter2019,EVI2winter2020,EVI2winter2021,EVI2winter2022,EVI2winter2023,GVMIsummer2018,GVMIsummer2019,GVMIsummer2020,GVMIsummer2021,GVMIsummer2022,GVMIsummer2023,GVMIwinter2018,GVMIwinter2019,GVMIwinter2020,GVMIwinter2021,GVMIwinter2022,GVMIwinter2023,MSVAIhypersummer2018,MSVAIhypersummer2019,MSVAIhypersummer2020,MSVAIhypersummer2021,MSVAIhypersummer2022,MSVAIhypersummer2023,MSVAIhyperwinter2018,MSVAIhyperwinter2019,MSVAIhyperwinter2020,MSVAIhyperwinter2021,MSVAIhyperwinter2022,MSVAIhyperwinter2023,MTVI2summer2018,MTVI2summer2019,MTVI2summer2020,MTVI2summer2021,MTVI2summer2022,MTVI2summer2023,MTVI2winter2018,MTVI2winter2019,MTVI2winter2020,MTVI2winter2021,MTVI2winter2022,MTVI2winter2023,MNDVIsummer2018,MNDVIsummer2019,MNDVIsummer2020,MNDVIsummer2021,MNDVIsummer2022,MNDVIsummer2023,MNDVIwinter2018,MNDVIwinter2019,MNDVIwinter2020,MNDVIwinter2021,MNDVIwinter2022,MNDVIwinter2023,OSAVIsummer2018,OSAVIsummer2019,OSAVIsummer2020,OSAVIsummer2021,OSAVIsummer2022,OSAVIsummer2023,OSAVIwinter2018,OSAVIwinter2019,OSAVIwinter2020,OSAVIwinter2021,OSAVIwinter2022,OSAVIwinter2023,PVIsummer2018,PVIsummer2019,PVIsummer2020,PVIsummer2021,PVIsummer2022,PVIsummer2023,PVIwinter2018,PVIwinter2019,PVIwinter2020,PVIwinter2021,PVIwinter2022,PVIwinter2023,SARVIsummer2018,SARVIsummer2019,SARVIsummer2020,SARVIsummer2021,SARVIsummer2022,SARVIsummer2023,SARVIwinter2018,SARVIwinter2019,SARVIwinter2020,SARVIwinter2021,SARVIwinter2022,SARVIwinter2023,SLAVIsummer2018,SLAVIsummer2019,SLAVIsummer2020,SLAVIsummer2021,SLAVIsummer2022,SLAVIsummer2023,SLAVIwinter2018,SLAVIwinter2019,SLAVIwinter2020,SLAVIwinter2021,SLAVIwinter2022,SLAVIwinter2023,TSAVI2summer2018,TSAVI2summer2019,TSAVI2summer2020,TSAVI2summer2021,TSAVI2summer2022,TSAVI2summer2023,TSAVI2winter2018,TSAVI2winter2019,TSAVI2winter2020,TSAVI2winter2021,TSAVI2winter2022,TSAVI2winter2023,WDVIsummer2018,WDVIsummer2019,WDVIsummer2020,WDVIsummer2021,WDVIsummer2022,WDVIsummer2023,WDVIwinter

In [19]:
for col in sent2_train_season_medians_wide.columns:
    print(pd.Series(sent2_train_season_medians_wide[col].isna().value_counts()))

long
False    148907
Name: count, dtype: int64
lat
False    148907
Name: count, dtype: int64
species_names
False    148907
Name: count, dtype: int64
B11summer2018
False    148907
Name: count, dtype: int64
B11summer2019
False    148907
Name: count, dtype: int64
B11summer2020
False    148907
Name: count, dtype: int64
B11summer2021
False    148907
Name: count, dtype: int64
B11summer2022
False    148907
Name: count, dtype: int64
B11summer2023
False    148907
Name: count, dtype: int64
B11winter2018
False    148907
Name: count, dtype: int64
B11winter2019
False    148907
Name: count, dtype: int64
B11winter2020
False    148907
Name: count, dtype: int64
B11winter2021
False    148907
Name: count, dtype: int64
B11winter2022
False    148907
Name: count, dtype: int64
B11winter2023
False    148907
Name: count, dtype: int64
B02summer2018
False    148907
Name: count, dtype: int64
B02summer2019
False    148907
Name: count, dtype: int64
B02summer2020
False    148907
Name: count, dtype: int64
B02summer20

##### Widen Test

In [20]:
sent2_test_season_medians_wide = sent2_test_season_medians.pivot(index=['long', 'lat', 'species_names'], columns=['season']).reset_index()
new_cols = ["".join(x) for x in sent2_test_season_medians_wide.columns]
sent2_test_season_medians_wide.columns = new_cols
sent2_test_season_medians_wide

,long,lat,species_names,B11summer2024,B11winter2024,B02summer2024,B02winter2024,B12summer2024,B12winter2024,B08summer2024,B08winter2024,B04summer2024,B04winter2024,B03summer2024,B03winter2024,NDVIsummer2024,NDVIwinter2024,ARVI2summer2024,ARVI2winter2024,BWDRVIsummer2024,BWDRVIwinter2024,CVIsummer2024,CVIwinter2024,CTVIsummer2024,CTVIwinter2024,EVI2summer2024,EVI2winter2024,GVMIsummer2024,GVMIwinter2024,MSVAIhypersummer2024,MSVAIhyperwinter2024,MTVI2summer2024,MTVI2winter2024,MNDVIsummer2024,MNDVIwinter2024,OSAVIsummer2024,OSAVIwinter2024,PVIsummer2024,PVIwinter2024,SARVIsummer2024,SARVIwinter2024,SLAVIsummer2024,SLAVIwinter2024,TSAVI2summer2024,TSAVI2winter2024,WDVIsummer2024,WDVIwinter2024,WDRVIsummer2024,WDRVIwinter2024
0,75.66669580000007,32.25107380000003,Bambusoideae spp.,0.29080,0.25830,0.14230,0.15780,0.22880,0.19190,0.33480,0.34770,0.16720,0.15900,0.1664,0.17030,0.002398,-0.034315,-0.177194,-0.220149,0.405837,0.375668,2.065170,1.906218,0.708800,0.682411,0.209016,0.272798,0.254991,0.354750,0.203063,0.263581,0.154652,0.239437,0.155412,0.285022,0.253745,0.328322,-0.765746,-0.752987,15.826980,20.746814,0.775898,1.000841,37.068751,28.242038,0.195003,0.228759,0.285961,0.372410
1,75.66676390000003,32.25073910000003,Bambusoideae spp.,0.31710,0.27550,0.13660,0.15180,0.24580,0.19890,0.34540,0.33450,0.15810,0.15340,0.1648,0.16380,-0.022566,-0.032787,-0.206402,-0.218361,0.421399,0.375694,2.035980,1.912466,0.690966,0.683530,0.253276,0.283844,0.221278,0.337335,0.245149,0.273260,0.204307,0.232773,0.133574,0.262741,0.305154,0.336195,-0.755262,-0.766043,19.636599,15.742295,0.799167,0.971819,29.353192,15.439635,0.218914,0.226362,0.345016,0.372159
2,75.66686810000004,32.25026770000005,Bambusoideae spp.,0.32890,0.28610,0.14420,0.15360,0.25470,0.20680,0.33860,0.37100,0.16220,0.15900,0.1720,0.17120,-0.020070,-0.031635,-0.203482,-0.217013,0.388559,0.414411,1.992846,2.012628,0.692770,0.684372,0.219832,0.302408,0.221538,0.348604,0.213420,0.292518,0.181254,0.267172,0.120755,0.282627,0.271958,0.356406,-0.761988,-0.729942,16.993831,34.403773,0.767839,1.027362,57.208149,23.117523,0.201435,0.252455,0.317657,0.400000
3,75.66719240000003,32.249825100000066,Bambusoideae spp.,0.32510,0.27620,0.14340,0.15570,0.25590,0.19410,0.32540,0.34350,0.15900,0.15620,0.1667,0.17140,-0.020333,-0.046398,-0.203789,-0.234286,0.364361,0.382651,1.984213,1.934911,0.692580,0.673500,0.206904,0.287520,0.214636,0.348844,0.198928,0.276591,0.177901,0.248658,0.112903,0.277902,0.261741,0.352054,-0.775044,-0.757141,13.576633,15.099199,0.752091,0.996998,31.677736,14.018858,0.188160,0.226670,0.305036,0.406310
4,75.66729090000007,32.24962370000003,Bambusoideae spp.,0.32770,0.27370,0.13640,0.15710,0.25180,0.19400,0.34350,0.33040,0.15520,0.14800,0.1603,0.16480,-0.016165,-0.040192,-0.198913,-0.227024,0.423539,0.424003,2.084338,2.123584,0.695583,0.678092,0.255823,0.298361,0.202223,0.341800,0.247436,0.286588,0.210158,0.255555,0.112010,0.266512,0.308859,0.367321,-0.757141,-0.770098,18.811617,14.103185,0.781214,1.019590,29.773170,11.618175,0.219119,0.228862,0.349971,0.426893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148902,78.12497830000007,31.25865170000003,Rhododendron arboreum,0.21160,0.17545,0.16735,0.12860,0.17215,0.14170,0.25400,0.20655,0.15155,0.14475,0.1650,0.13740,-0.046512,-0.000522,-0.234420,-0.180610,0.194441,0.223169,1.435463,1.551780,0.673342,0.706637,0.156266,0.092444,0.302340,0.305836,0.149567,0.087003,0.144799,0.073740,0.196189,0.179411,0.200646,0.131142,-0.845664,-0.892596,5.744313,5.542639,0.775235,0.708139,9.573166,7.617956,0.140034,0.093198,0.237574,0.165224
148903,78.12522000000007,31.258880000000033,Rhododendron arboreum,0.22900,0.17825,0.17680,0.12045,0.18470,0.14705,0.29230,0.19560,0.15390,0.15065,0.1825,0.14280,-0.028933,-0.012505,-0.213851,-0.194631,0.198402,0.228580,1.487546,1.417172,0.686226,0.698096,0.1630

In [21]:
for col in sent2_test_season_medians_wide.columns:
    print(pd.Series(sent2_test_season_medians_wide[col].isna().value_counts()))

long
False    148907
Name: count, dtype: int64
lat
False    148907
Name: count, dtype: int64
species_names
False    148907
Name: count, dtype: int64
B11summer2024
False    148907
Name: count, dtype: int64
B11winter2024
False    148907
Name: count, dtype: int64
B02summer2024
False    148907
Name: count, dtype: int64
B02winter2024
False    148907
Name: count, dtype: int64
B12summer2024
False    148907
Name: count, dtype: int64
B12winter2024
False    148907
Name: count, dtype: int64
B08summer2024
False    148907
Name: count, dtype: int64
B08winter2024
False    148907
Name: count, dtype: int64
B04summer2024
False    148907
Name: count, dtype: int64
B04winter2024
False    148907
Name: count, dtype: int64
B03summer2024
False    148907
Name: count, dtype: int64
B03winter2024
False    148907
Name: count, dtype: int64
NDVIsummer2024
False    148907
Name: count, dtype: int64
NDVIwinter2024
False    148907
Name: count, dtype: int64
ARVI2summer2024
False    148907
Name: count, dtype: int64
ARVI2wi

### Exporting Frames

In [22]:
sent_2_month_medians_path = processed_folder.joinpath("sent_2_month_medians.parquet")
sent_2_season_medians_path = processed_folder.joinpath("sent_2_season_medians.parquet")

sent2_train_season_medians_widened_path = processed_folder.joinpath("sent2_train_season_medians_widened.parquet")
sent2_test_season_medians_widened_path = processed_folder.joinpath("sent2_test_season_medians_widened.parquet")

In [23]:
df_month_medians.to_parquet(sent_2_month_medians_path, index=False)
df_season_medians.to_parquet(sent_2_season_medians_path, index=False)

In [24]:
sent2_train_season_medians_wide.to_parquet(sent2_train_season_medians_widened_path, index=False)
sent2_test_season_medians_wide.to_parquet(sent2_test_season_medians_widened_path, index=False)